<a href="https://colab.research.google.com/github/nargesalavi/Quantum-Open-Source-Foundation-Mentorship/blob/master/QNN_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.1.0

In [ ]:
!pip install tensorflow-quantum

In [1]:
import tensorflow as tf
import tensorflow_quantum as tfq
import cirq
import sympy
import numpy as np

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
qubits = cirq.GridQubit.rect(1, 4)

In [3]:
def generate_even_block(block_number):
    """ Function for generating the even blocks
        Arguments:
          block_number: Block number, it has to be even.
        return: ciq.Circuit """
    params = sympy.symbols(['theta_{}'.format(n) for n in range((block_number-1)*4,block_number*4)])

    # create the parameterized circuit
    circuit = cirq.Circuit(
        cirq.rz(params[0])(qubits[0]),
        cirq.rz(params[1])(qubits[1]),
        cirq.rz(params[2])(qubits[2]),
        cirq.rz(params[3])(qubits[3]),
        cirq.CZ(qubits[0],qubits[1]),
        cirq.CZ(qubits[0],qubits[2]),
        cirq.CZ(qubits[0],qubits[3]),
        cirq.CZ(qubits[1],qubits[2]),
        cirq.CZ(qubits[1],qubits[3]),
        cirq.CZ(qubits[2],qubits[3])
    )
    
    return circuit

In [4]:
def generate_odd_block(block_number):
    """ Function for generating the odd blocks
        block_number: Block number, it has to be odd.
        return: ciq.Circuit """
    params = sympy.symbols(['theta_{}'.format(n) for n in range((block_number-1)*4,block_number*4)])

    # create the parameterized circuit
    circuit = cirq.Circuit(
        cirq.rx(params[0])(qubits[0]),
        cirq.rx(params[1])(qubits[1]),
        cirq.rx(params[2])(qubits[2]),
        cirq.rx(params[3])(qubits[3])
    )
    
    return circuit

In [5]:
def generate_qnn(l):
    """ Function for generating qnn, containing l number of layers.
        Arguments:
          l: number of layers, each layer contains one odd and one even block.
        return: ciq.Circuit """
    circuit = cirq.Circuit()
    for i in range(1,2*l+1):
        if i % 2 == 1:
            circuit += generate_odd_block(i)
        else:
            circuit += generate_even_block(i)
        
    return circuit

In [6]:
l=1
qnn = generate_qnn(l)
print(qnn)

                                               ┌──┐
(0, 0): ───Rx(theta_0)───Rz(theta_4)───@───@────@─────────────
                                       │   │    │
(0, 1): ───Rx(theta_1)───Rz(theta_5)───@───┼────┼@────@───────
                                           │    ││    │
(0, 2): ───Rx(theta_2)───Rz(theta_6)───────@────┼@────┼───@───
                                                │     │   │
(0, 3): ───Rx(theta_3)───Rz(theta_7)────────────@─────@───@───
                                               └──┘


In [7]:
params = tf.random.uniform(shape = (1,8*l), minval=0, maxval=2*np.pi) #tf.zeros([1,8*l ], tf.float32)
print(params)
params_names = sympy.symbols(['theta_{}'.format(n) for n in range(8*l)])

tf.Tensor(
[[3.486247   6.187014   4.490873   4.0299993  5.8754673  2.493145
  5.2590923  0.25225595]], shape=(1, 8), dtype=float32)


In [8]:
state_layer = tfq.layers.State()
state = state_layer(qnn, symbol_names=params_names, symbol_values=params)
print(qnn)
print(state)

Instructions for updating:
reduction_indices is deprecated, use axis instead
                                               ┌──┐
(0, 0): ───Rx(theta_0)───Rz(theta_4)───@───@────@─────────────
                                       │   │    │
(0, 1): ───Rx(theta_1)───Rz(theta_5)───@───┼────┼@────@───────
                                           │    ││    │
(0, 2): ───Rx(theta_2)───Rz(theta_6)───────@────┼@────┼───@───
                                                │     │   │
(0, 3): ───Rx(theta_3)───Rz(theta_7)────────────@─────@───@───
                                               └──┘
<tf.RaggedTensor [[(0.03640970215201378-0.028071235865354538j), (0.03802140802145004+0.08880311250686646j), (0.05713210627436638-0.006315305829048157j), (0.017112167552113533-0.11955693364143372j), (-0.0021348693408071995-0.0005806349217891693j), (-0.0023009879514575005+0.004039253108203411j), (0.0019026931840926409+0.0020077480003237724j), (-0.0050829327665269375+0.002818431705236435j), (0.2310047

In [9]:
random_circuit = cirq.testing.random_circuit(qubits = qubits,n_moments = np.random.randint(low=1,high=5),\
                                                                op_density = 0.99999999)
print(random_circuit)
target_state = tfq.layers.State()(random_circuit)
print(target_state)

(0, 0): ───T───S───Y───────@───
                           │
(0, 1): ───────@───────────@───
               │
(0, 2): ───@───@───iSwap───────
           │       │
(0, 3): ───@───T───iSwap───Z───
<tf.RaggedTensor [[-4.371138828673793e-08j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-4.371138828673793e-08+1j), 0j, 0j, 0j, 0j, 0j, 0j, 0j]]>


In [64]:
def states_distance(state,target):
  diff = state[0] - target[0]
  diff = tf.reshape(diff, [16, 1])
  conjugate_transposed_state = tf.transpose(diff,conjugate=True)
  distance = tf.tensordot(conjugate_transposed_state, diff, axes = 1)
  #print('distance:' , distance, 'state:', state)
  return tf.dtypes.cast(distance, tf.float32)

In [67]:
a = states_distance(state,target_state)
print(a)

tf.Tensor([[1.7438209]], shape=(1, 1), dtype=float32)


In [69]:
param_length = len(params[0])
print(params)

tf.Tensor(
[[3.486247   6.187014   4.490873   4.0299993  5.8754673  2.493145
  5.2590923  0.25225595]], shape=(1, 8), dtype=float32)


In [65]:
# gradient(theta[i]) = function(theta)
def custom_gradient(params, target_state, parameter_shift = True, h = np.pi/2):
  """ This function calculates the gradients of the quantum circuit's parameters 
      using two methods: Parameter shift rules or Numerical differentiation.
      In numerical differentiation method, for Pauli gates the shift value is pi/2 and the ratio is 1/2. 
      Arguments:
        params: circuit's parameters, of which the gradient is calculated.
        target_state: The target state, which the quantum NN trys to approximate.
        parameter_shift: Determine which method is used to calculate the gradients. 
                         When it is True, the function uses parameter shift otherwise it uses numerical differentiation.
        h: The shift value for parameter. When parameter shift method is used, it is pi/2."""
  param_length = len(params[0])
  grad = [None]*param_length
  if parameter_shift : 
    r = 1/2
    h = np.pi/2
  else:
    r = 1/(2*h)
  for i in range(param_length):
    
    params_plus_h = tf.convert_to_tensor([np.concatenate((params[0][0:i],[params[0][i]+h],params[0][i+1:]))])
    params_minus_h = tf.convert_to_tensor([np.concatenate((params[0][0:i],[params[0][i]-h],params[0][i+1:]))])
    state_plus_h = state_layer(qnn, symbol_names=params_names, symbol_values=params_plus_h)
    state_minus_h = state_layer(qnn, symbol_names=params_names, symbol_values=params_minus_h)
    #print('i', i, 'state_plus_h:', state_plus_h, 'state_minus_h:', state_minus_h) 
    grad[i] = r*(states_distance(state_plus_h,target_state)-states_distance(state_minus_h,target_state))

  return grad

In [73]:
def update_params(params, gradients, learning_rate):
  print('1:', gradients)
  gradients = tf.reshape(gradients,[1,8])
  print('2:', gradients)
  print('3:', params)
  params -= learning_rate*gradients
  params = params % (2*np.pi)
  print('4:', params)
  return params


In [71]:
print(params)
params_grads = custom_gradient(params, target_state)

tf.Tensor(
[[3.486247   6.187014   4.490873   4.0299993  5.8754673  2.493145
  5.2590923  0.25225595]], shape=(1, 8), dtype=float32)


In [74]:
update_params(params, params_grads,0.01)

1: [<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.03152907]], dtype=float32)>, <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.00871736]], dtype=float32)>, <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.22647953]], dtype=float32)>, <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.38061833]], dtype=float32)>, <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.32669002]], dtype=float32)>, <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.32669002]], dtype=float32)>, <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.32669002]], dtype=float32)>, <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.3266899]], dtype=float32)>]
2: tf.Tensor(
[[ 0.03152907 -0.00871736 -0.22647953 -0.38061833 -0.32669002  0.32669002
   0.32669002  0.3266899 ]], shape=(1, 8), dtype=float32)
3: tf.Tensor(
[[3.486247   6.187014   4.490873   4.0299993  5.8754673  2.493145
  5.2590923  0.25225595]], shape=(1, 8), dtype=float32)
4: tf.Tensor(
[[3.48593

<tf.Tensor: shape=(1, 8), dtype=float32, numpy=
array([[3.4859319 , 6.1871014 , 4.493138  , 4.0338054 , 5.878734  ,
        2.4898782 , 5.2558255 , 0.24898905]], dtype=float32)>

In [35]:
tf.shape(params)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 8], dtype=int32)>

In [44]:
a = tf.keras.initializers.Zeros()(shape = (1,8))
print(a[0][1])

tf.Tensor(0.0, shape=(), dtype=float32)


AttributeError: ignored